In [1]:
from load_notebook_config import LoadDirectoriesConfig, LoadRAGConfig, LoadOpenAIConfig, LoadConfig
from langchain_core.messages import ToolMessage
from utilities import _print_event

In [2]:
CFG_DIRECTORIES = LoadDirectoriesConfig()
CFG_OPENAI = LoadOpenAIConfig()
CFG_RAG = LoadRAGConfig()
CFG = LoadConfig()
db = CFG_DIRECTORIES.local_file
backup_file = CFG_DIRECTORIES.backup_file

In [3]:
from construct_graph import BrookAIGraph
graph_instance = BrookAIGraph()
graph = graph_instance.Compile_graph()

In [15]:
# from IPython.display import Image

# # Generate the image
# try:
#     image = part_4_graph.get_graph(xray=True).draw_mermaid_png()
    
#     # Save the image to a file
#     with open("Brook_AI.png", "wb") as file:
#         file.write(image)
        
#     # Display the image
#     display(Image("Brook_AI.png"))
# except Exception as e:
#     print("An error occurred:", e)

In [4]:
sample_questions = [
    "Hi there, what time is my flight?",
    "Am i allowed to update my flight to something sooner? I want to leave later today.",
    "Update my flight to sometime next week then",
    "The next available option is great",
    "what about lodging and transportation?",
    "I'd like an affordable hotel for my week-long stay (7 days). And I'll want to rent a car.",
    # "OK could you place a reservation for your recommended hotel? It sounds nice.",
    # "yes go ahead and book anything that's moderate expense and has availability.",
    # "Now for a car, what are my options?",
    # "Awesome let's just get the cheapest option. Go ahead and book for 7 days",
    # "Cool so now what recommendations do you have on excursions?",
    # "Are they available while I'm there?",
    # "interesting - i like the museums, what options are there? ",
    # "OK great pick one and book it for my second day there.",
]

In [5]:
import shutil
import uuid


# Update with the backup file so we can restart from the original place in each section
shutil.copy(backup_file, db)
thread_id = str(uuid.uuid4())
print("thread_id:", thread_id)

config = {
    "configurable": {
        # The passenger_id is used in our flight tools to
        # fetch the user's flight information
        "passenger_id": "3442 587242",
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
        "recursion_limit": 50
    }
}

_printed = set()
# We can reuse the tutorial questions from part 1 to see how it does.
for question in sample_questions:
    events = graph.stream(
        {"messages": ("user", question)}, config, stream_mode="values"
    )
    for event in events:
        _print_event(event, _printed)
    snapshot = graph.get_state(config)
    while snapshot.next:
        # We have an interrupt! The agent is trying to use a tool, and the user can approve or deny it
        # Note: This code is all outside of your graph. Typically, you would stream the output to a UI.
        # Then, you would have the frontend trigger a new run via an API call when the user has provided input.
        user_input = input(
            "Do you approve of the above actions? Type 'y' to continue;"
            " otherwise, explain your requested changed.\n\n"
        )
        if user_input.strip() == "y":
            # Just continue
            result = graph.invoke(
                None,
                config,
            )
        else:
            # Satisfy the tool invocation by
            # providing instructions on the requested changes / change of mind
            result = graph.invoke(
                {
                    "messages": [
                        ToolMessage(
                            tool_call_id=event["messages"][-1].tool_calls[0]["id"],
                            content=f"API call denied by user. Reasoning: '{user_input}'. Continue assisting, accounting for the user's input.",
                        )
                    ]
                },
                config,
            )
        snapshot = graph.get_state(config)

thread_id: 5579d427-0f29-451b-8a14-211338300e26
================================ Human Message =================================

Hi there, what time is my flight?
================================== Ai Message ==================================

Your flight, LX0112, is scheduled to depart from Charles de Gaulle Airport (CDG) on April 30, 2024, at 12:09 PM local time and is set to arrive at EuroAirport Basel Mulhouse Freiburg (BSL) at 1:39 PM local time. Please make sure to check for any updates closer to your departure date.
================================ Human Message =================================

Am i allowed to update my flight to something sooner? I want to leave later today.
================================== Ai Message ==================================
Tool Calls:
  lookup_policy (call_YJjwdXHFoXSGTnQ0rn6K6VtK)
 Call ID: call_YJjwdXHFoXSGTnQ0rn6K6VtK
  Args:
    query: flight change policy
================================= Tool Message =================================
Na

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_uRHGd29kpQO8ptxfXkm1NzZr", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

In [7]:
snapshot[0]["messages"][-1]

AIMessage(content='I have successfully booked the Holiday Inn Basel for your week-long stay in Basel. If you need any further assistance or changes, feel free to let me know!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 1736, 'total_tokens': 1769}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-caebb8ae-3d72-4195-94ba-ce73aef89d94-0', usage_metadata={'input_tokens': 1736, 'output_tokens': 33, 'total_tokens': 1769})

In [10]:
import shutil
import uuid

# Update with the backup file so we can restart from the original place in each section
shutil.copy(backup_file, db)
thread_id = str(uuid.uuid4())
print("thread_id:", thread_id)

config = {
    "configurable": {
        # The passenger_id is used in our flight tools to
        # fetch the user's flight information
        "passenger_id": "3442 587242",
        # Checkpoints are accessed by thread_id
        "thread_id": thread_id,
    }
}

_printed = set()

thread_id: 6aa90ec2-aad3-4cef-b092-adab131b0f89


In [14]:
question = "Hi there, what time is my flight?"
question = "Am i allowed to update my flight to something sooner? I want to leave later today."
# question = "Update my flight to sometime next week then"
# question =  "The next available option is great"
# question =  "what about lodging and transportation?"
    # "Yeah i think i'd like an affordable hotel for my week-long stay (7 days). And I'll want to rent a car.",
    # "OK could you place a reservation for your recommended hotel? It sounds nice.",
    # "yes go ahead and book anything that's moderate expense and has availability.",
    # "Now for a car, what are my options?",
    # "Awesome let's just get the cheapest option. Go ahead and book for 7 days",
    # "Cool so now what recommendations do you have on excursions?",
    # "Are they available while I'm there?",
    # "interesting - i like the museums, what options are there? ",
    # "OK great pick one and book it for my second day there.",
# ]

In [15]:
events = graph.stream(
    {"messages": ("user", question)}, config, stream_mode="values"
)
for event in events:
    _print_event(event, _printed)
snapshot = graph.get_state(config)
while snapshot.next:
    # We have an interrupt! The agent is trying to use a tool, and the user can approve or deny it
    # Note: This code is all outside of your graph. Typically, you would stream the output to a UI.
    # Then, you would have the frontend trigger a new run via an API call when the user has provided input.
    user_input = input(
        "Do you approve of the above actions? Type 'y' to continue;"
        " otherwise, explain your requested changed.\n\n"
    )
    if user_input.strip() == "y":
        # Just continue
        result = graph.invoke(
            None,
            config,
        )
    else:
        # Satisfy the tool invocation by
        # providing instructions on the requested changes / change of mind
        result = graph.invoke(
            {
                "messages": [
                    ToolMessage(
                        tool_call_id=event["messages"][-1].tool_calls[0]["id"],
                        content=f"API call denied by user. Reasoning: '{user_input}'. Continue assisting, accounting for the user's input.",
                    )
                ]
            },
            config,
        )
    snapshot = graph.get_state(config)

================================ Human Message =================================

Am i allowed to update my flight to something sooner? I want to leave later today.
================================== Ai Message ==================================
Tool Calls:
  lookup_policy (call_2ODnzgeW2YWUTexdvIvtVhp7)
 Call ID: call_2ODnzgeW2YWUTexdvIvtVhp7
  Args:
    query: change flight date
================================= Tool Message =================================
Name: lookup_policy


## Booking and Cancellation

1. How can I change my booking?
	* The ticket number must start with 724 (SWISS ticket no./plate).
	* The ticket was not paid for by barter or voucher (there are exceptions to voucher payments; if the ticket was paid for in full by voucher, then it may be possible to rebook online under certain circumstances. If it is not possible to rebook online because of the payment method, then you will be informed accordingly during the rebooking process).
	* There must be an active flight 

In [16]:
event

{'messages': [HumanMessage(content='Hi there, what time is my flight?', id='897103d0-a3a2-4186-8e2c-0cbd8862763d'),
  AIMessage(content='Your flight details are as follows:\n- **Flight Number:** LX0112\n- **Departure Airport:** CDG\n- **Arrival Airport:** BSL\n- **Scheduled Departure:** 2024-04-30 12:09:03.561731-04:00\n- **Scheduled Arrival:** 2024-04-30 13:39:03.561731-04:00\n- **Seat Number:** 18E\n- **Fare Conditions:** Economy\n\nIf you need any further assistance or information, feel free to let me know!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 121, 'prompt_tokens': 1009, 'total_tokens': 1130}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-03a439b5-157b-4b0b-929f-7de8e784a4c6-0', usage_metadata={'input_tokens': 1009, 'output_tokens': 121, 'total_tokens': 1130}),
  HumanMessage(content='Am i allowed to update my flight to something sooner? I want to leave lat

In [60]:
snapshot[0]["messages"]

[HumanMessage(content='Hi there, what time is my flight?', id='9c5d9b25-78e4-42e0-8372-b9fc79976dcc'),
 AIMessage(content='Your flight with ticket number 7240005432906569 is scheduled as follows:\n\n- **Flight Number:** LX0112\n- **Departure Airport:** CDG (Paris Charles de Gaulle Airport)\n- **Arrival Airport:** BSL (EuroAirport Basel Mulhouse Freiburg)\n- **Scheduled Departure:** April 30, 2024, at 12:09 PM (CDT)\n- **Scheduled Arrival:** April 30, 2024, at 01:39 PM (CDT)\n- **Seat Number:** 18E\n- **Fare Conditions:** Economy\n\nIf you need any further assistance or information, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 139, 'prompt_tokens': 1009, 'total_tokens': 1148}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b8447700-5085-43cb-b633-129f6027ffe2-0', usage_metadata={'input_tokens': 1009, 'output_tokens': 139, 'total_tokens': 1148}),
 Human

In [63]:
snapshot[0]["messages"]

[HumanMessage(content='Hi there, what time is my flight?', id='9c5d9b25-78e4-42e0-8372-b9fc79976dcc'),
 AIMessage(content='Your flight with ticket number 7240005432906569 is scheduled as follows:\n\n- **Flight Number:** LX0112\n- **Departure Airport:** CDG (Paris Charles de Gaulle Airport)\n- **Arrival Airport:** BSL (EuroAirport Basel Mulhouse Freiburg)\n- **Scheduled Departure:** April 30, 2024, at 12:09 PM (CDT)\n- **Scheduled Arrival:** April 30, 2024, at 01:39 PM (CDT)\n- **Seat Number:** 18E\n- **Fare Conditions:** Economy\n\nIf you need any further assistance or information, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 139, 'prompt_tokens': 1009, 'total_tokens': 1148}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b8447700-5085-43cb-b633-129f6027ffe2-0', usage_metadata={'input_tokens': 1009, 'output_tokens': 139, 'total_tokens': 1148}),
 Human

In [70]:
for event in events:
    _print_event(event, _printed)

In [71]:
event

{'messages': [HumanMessage(content='Hi there, what time is my flight?', id='9c5d9b25-78e4-42e0-8372-b9fc79976dcc'),
  AIMessage(content='Your flight with ticket number 7240005432906569 is scheduled as follows:\n\n- **Flight Number:** LX0112\n- **Departure Airport:** CDG (Paris Charles de Gaulle Airport)\n- **Arrival Airport:** BSL (EuroAirport Basel Mulhouse Freiburg)\n- **Scheduled Departure:** April 30, 2024, at 12:09 PM (CDT)\n- **Scheduled Arrival:** April 30, 2024, at 01:39 PM (CDT)\n- **Seat Number:** 18E\n- **Fare Conditions:** Economy\n\nIf you need any further assistance or information, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 139, 'prompt_tokens': 1009, 'total_tokens': 1148}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b8447700-5085-43cb-b633-129f6027ffe2-0', usage_metadata={'input_tokens': 1009, 'output_tokens': 139, 'total_tokens': 

In [65]:
def requires_approval(actions_list):
    if not actions_list:
        return False

    # Get the last item in the list
    last_action = actions_list[-1]
    
    # Check if the last item is a list and has at least two elements
    if isinstance(last_action, list) and len(last_action) > 1:
        # Check if the second element contains the approval prompt
        approval_prompt = "Do you approve of the above actions? Type 'y' to continue. Otherwise, explain your requested changed."
        return approval_prompt in last_action[1]
    
    return False

# Example usage
actions_list = [
    ['Update my flight to sometime next week', "I'm sorry, but there are currently no available flights for next week. If you have any other preferences or questions, feel free to let me know how else I can assist you."],
    ['cancel my flight', "Do you approve of the above actions? Type 'y' to continue. Otherwise, explain your requested changed."]
]

print(requires_approval(actions_list))  # Output: True

True
